In [16]:
import os
import yaml

# Load configuration
config = yaml.safe_load(open("myconfig.yml"))

# Set up Azure OpenAI environment variables with the minimum necessary configuration
os.environ["AZURE_OPENAI_API_KEY"] = config["AZURE_OPENAI_API_KEY"]
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = config["AZURE_OPENAI_DEPLOYMENT_NAME"]
os.environ["AZURE_OPENAI_ENDPOINT"] = config["AZURE_OPENAI_ENDPOINT"]
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-05-15"  # Using a stable API version that works with most Azure OpenAI deployments

os.environ["TAVILY_API_KEY"] = config["TAVILY_API_KEY"]
os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["LANGCHAIN_HUB_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = str(config["LANGCHAIN_TRACING_V2"]).lower()
os.environ["LANGCHAIN_ENDPOINT"] = config["LANGCHAIN_ENDPOINT"]
os.environ["LANGCHAIN_HUB_API_URL"] = config["LANGCHAIN_HUB_API_URL"]
os.environ["LANGCHAIN_WANDB_TRACING"] = str(config["LANGCHAIN_WANDB_TRACING"]).lower()
os.environ["WANDB_PROJECT"] = str(config["WANDB_PROJECT"])

# Planning

![images/Screenshot 2024-02-21 at 16.17.22.png](<images/Screenshot 2024-02-21 at 16.17.22.png>)

You might have come across various techniques aimed at improving the performance of large language models, such as offering tips or even jokingly threatening them. One popular technique is called "chain of thought," where the model is asked to think step by step, enabling self-correction. This approach has evolved into more advanced versions like "chain of thought with self-consistency" and the generalized "tree of thoughts," where multiple thoughts are created, re-evaluated, and consolidated to provide an output.

### Tree of Thoughts

The [@astropomeai tutorial](https://medium.com/@astropomeai/implementing-the-tree-of-thoughts-in-langchains-chain-f2ebc5864fac) on Tree of Thoughts is used as basis of this exercise but expanded with LLMOps tools.

In [17]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import AzureChatOpenAI

In [18]:
from langchain import hub

cot_step1 = hub.pull("rachnogstyle/nlw_jan24_cot_step1")
cot_step2 = hub.pull("rachnogstyle/nlw_jan24_cot_step2")
cot_step3 = hub.pull("rachnogstyle/nlw_jan24_cot_step3")
cot_step4 = hub.pull("rachnogstyle/nlw_jan24_cot_step4")

In [19]:
from langchain_community.chat_models import AzureChatOpenAI

# Get Azure OpenAI configuration from environment variables
deployment_name = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
api_version = os.environ["AZURE_OPENAI_API_VERSION"]
api_key = os.environ["AZURE_OPENAI_API_KEY"]

azure_llm = AzureChatOpenAI(
    deployment_name=deployment_name,
    api_version=api_version,
    api_key=api_key,
    temperature=0
)

chain1 = LLMChain(
    llm=azure_llm,
    prompt=cot_step1,
    output_key="solutions"
)

chain2 = LLMChain(
    llm=azure_llm,
    prompt=cot_step2,
    output_key="review"
)

chain3 = LLMChain(
    llm=azure_llm,
    prompt=cot_step3,
    output_key="deepen_thought_process"
)

chain4 = LLMChain(
    llm=azure_llm,
    prompt=cot_step4,
    output_key="ranked_solutions"
)

In [21]:
from langchain.chains import SequentialChain

# Set up the chain with the Azure OpenAI LLM
overall_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["input", "perfect_factors"],
    output_variables=["ranked_solutions"],
    verbose=True
)

print(
        overall_chain.invoke(
            {
                "input": "human colonization of Mars",
                "perfect_factors": "The distance between Earth and Mars is very large, making regular resupply difficult"
            }
    )
)

#print(
#        overall_chain.invoke(
#            {
#                "input": "Father teaching his son how to play chess",
#                "perfect_factors": "The father is a SRE, while the son is a 8 year old child"
#            }
#        )
#)



> Entering new SequentialChain chain...

> Finished chain.
{'input': 'human colonization of Mars', 'perfect_factors': 'The distance between Earth and Mars is very large, making regular resupply difficult', 'ranked_solutions': "### **Ranking of Solutions**\n\n1. **Solution 2: Modular Supply Chain with Orbital Depots**  \n   **Justification:**  \n   This solution ranks highest due to its practicality, scalability, and ability to address immediate needs for a Martian colony. Establishing a reliable supply chain ensures that colonists have access to essential resources (food, water, equipment) during the early stages of settlement, when self-sufficiency is not yet achievable. The modular nature of the depots allows for incremental expansion and adaptation to growing demands. Additionally, this approach leverages existing technologies (e.g., reusable spacecraft) and partnerships with aerospace companies, making it the most feasible in the short term.  \n   **Final Thoughts:**  \n   While 

### ReAct prompt overview

Let's review [ReAct](https://python.langchain.com/docs/modules/agents/agent_types/react) prompt as it's defined in Langchain.

In [22]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")
prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

### Self-ask with search

Let's review [self-ask with search](https://python.langchain.com/docs/modules/agents/agent_types/self_ask_with_search) as it's defined in Langchain.

In [23]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/self-ask-with-search")
prompt.template

'Question: Who lived longer, Muhammad Ali or Alan Turing?\nAre follow up questions needed here: Yes.\nFollow up: How old was Muhammad Ali when he died?\nIntermediate answer: Muhammad Ali was 74 years old when he died.\nFollow up: How old was Alan Turing when he died?\nIntermediate answer: Alan Turing was 41 years old when he died.\nSo the final answer is: Muhammad Ali\n\nQuestion: When was the founder of craigslist born?\nAre follow up questions needed here: Yes.\nFollow up: Who was the founder of craigslist?\nIntermediate answer: Craigslist was founded by Craig Newmark.\nFollow up: When was Craig Newmark born?\nIntermediate answer: Craig Newmark was born on December 6, 1952.\nSo the final answer is: December 6, 1952\n\nQuestion: Who was the maternal grandfather of George Washington?\nAre follow up questions needed here: Yes.\nFollow up: Who was the mother of George Washington?\nIntermediate answer: The mother of George Washington was Mary Ball Washington.\nFollow up: Who was the fathe

Approaches to read next:

**Reflexion** ([Shinn & Labash 2023](https://arxiv.org/abs/2303.11366)) is a framework to equips agents with dynamic memory and self-reflection capabilities to improve reasoning skills.

**Chain of Hindsight** (CoH; [Liu et al. 2023](https://arxiv.org/abs/2302.02676)) encourages the model to improve on its own outputs by explicitly presenting it with a sequence of past outputs, each annotated with feedback.